# Discounting Cashflows with low memory utilization

In the lifelib codebase there is code like

```py
def pv_commissions():
    result = np.array(list(commissions(t) for t in range(max_proj_len()))).transpose()
    return result @ disc_factors()[:max_proj_len()]
```

* `result` is shape (Policies, Timesteps)
    * Shape (Timesteps, Policies): np.array(list(commissions(t) for t in range(max_proj_len())))
    * Shape (Policies, Timesteps): after transposing
* `disc_factors()[:max_proj_len()]` is shape (timesteps)

This first section discusses if matrix multiplications that allocate an array of (P, T) cashflows are necessary for performant discounting.

In [1]:
import numpy as np
from functools import lru_cache

policy_count = 10000
policy_term = 20
policy_timesteps = 12 * policy_term + 1
mortality_rates = np.random.uniform(0, 0.01, size=(policy_timesteps, policy_count))
discount_factors = np.linspace(0.99, 0.6, policy_timesteps)

@lru_cache(maxsize=1)
def pols_if(t):
    if t == 0:
        return np.ones(policy_count)
    return pols_if(t-1) * (1 - mortality_rates[t-1])

def pv_matmul():
    result = np.array(list(pols_if(t) for t in range(policy_timesteps))).transpose()
    return result @ discount_factors

def pv_loop():
    return sum(pols_if(t) * discount_factors[t] for t in range(policy_timesteps))

We find that doing a loop over the timesteps is faster than a matrix multiplication.

In [2]:
%%timeit
pv_matmul()


22.2 ms ± 3.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
%%timeit
pv_loop()

3.98 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Optimizing BasicTerm_M from O(P*T) to O(P) memory with an LRU cache

In [4]:
from Cash import cash
from lru import LRU
from model1 import net_premium_pp as net_premium_pp1, pv_net_cf as pv_net_cf1
from model2 import net_premium_pp as net_premium_pp2, pv_net_cf as pv_net_cf2
from model3 import net_premium_pp as net_premium_pp3, pv_net_cf as pv_net_cf3

## Net premium

We would like to minimize cache misses to improve performance, so we do a slight refactor:

This code creates two cache misses for each call to `pols_if(t)`
```py
@cash
def net_premium_pp():
    return pv_claims() / pv_pols_if()
```
This code has one cache miss for each call to `pols_if(t)`
```py
@cash
def net_premium_pp():
    numerator = 0
    denominator = 0
    for t in range(max_proj_len):
        denominator += pols_if(t) * discount(t)
        numerator += claims(t) * discount(t)
    return numerator / denominator
```

In [5]:
%%timeit
# Cache does not evict
cash.reset(dict)
net_premium_pp1()

318 ms ± 32.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
# LRU Cache without code modifications to optimize cache misses
cash.reset(lambda: LRU(1))
net_premium_pp1()

440 ms ± 43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
# LRU Cache with code modifications to optimize cache misses
cash.reset(lambda: LRU(1))
net_premium_pp2()

335 ms ± 85.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## PV Net CF

When using an LRU cache with a max size of 1, logic like this will prevent exponential time blowup, but still causes undesirable cache misses.

```py
@cash
def pv_net_cf():
    return pv_premiums() - pv_claims() - pv_expenses() - pv_commissions()
```

Consider that after we calculate pv_premiums, `pols_if(max_proj_len-1)` will be the only cached value for `pols_if`. When we calculate `pv_claims`, `pv_expenses`, and `pv_commissions`, all calls to `pols_if` will be cache misses.

```py
...
@cash
def pv_premiums():
    return sum(premiums(t) * discount(t) for t in range(max_proj_len))
@cash
def pv_claims():
    return sum(claims(t) * discount(t) for t in range(max_proj_len))
...
```

See below that the LRU cache causes major performance degradations.

In [8]:
%%timeit
# Cache does not evict
cash.reset(dict)
pv_net_cf1()

515 ms ± 46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
# LRU Cache without code modifications to optimize cache misses
cash.reset(lambda: LRU(1))
pv_net_cf1()

1.16 s ± 82.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can avoid this situation with a slight refactoring - 

```py
@cash
def net_cf(t):
    return premiums(t) - claims(t) - expenses(t) - commissions(t)
@cash
def pv_net_cf():
    return sum(net_cf(t) * discount(t) for t in range(max_proj_len))
```

In [10]:
%%timeit
# LRU Cache with code modifications to optimize cache misses
cash.reset(lambda: LRU(1))
pv_net_cf3()

680 ms ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The non-evicting cache `cache.reset(dict)` has only one cache miss for each function. The LRU cache even with the optimizations discussed will have two cache misses as two passes are required. One pass for the net premium calculation and one pass for the net cashflows.

Despite these cache misses, it is not **twice** as slow, which is was what I was expecting.

## Memory utilization

The LRU(1) should keep the calculations to $O(P)$ memory complexity, which we expect to reduce memory consumption by a factor of 1000 for models with 1000 timesteps.

TODO: Run experiments and get results for actual memory utilization.